In [ ]:
import pandas as pd
import operator
import time
import numpy as np

In [ ]:
config = {}

config["project_path"] = ".."
config["data_folder_path"] = "data"
config["temp_folder_path"] = "temp"

# input files
config['channel_input'] = 'Channel AnP 202001-05.xlsx'

# output files
config['channel_output'] = 'Channel AnP split 202001-04.xlsx'

## Read Channel Input

In [ ]:
input_file_path = config['project_path'] + '/' \
                        + config['data_folder_path'] + '/' \
                        + config['channel_input']

input_df = pd.read_excel(input_file_path, dtype=str)
input_df['Value'] = input_df['Spending_Value'].astype('float')

In [ ]:
channel_df = input_df.groupby(by = ['Date', 'Brand', 'Province', 'Cost_Type_2'])['Value'].sum().reset_index()

#### Create all year, province, KPI combination for 2020

In [ ]:
all_dates_map = pd.DataFrame(columns=['Date','Brand', 'Province', 'Cost_Type_2'])
all_types= channel_df[['Brand', 'Province', 'Cost_Type_2']].drop_duplicates()
for i in range(1,13):
    year_month = "2020" + str(i).zfill(2)
    all_types['Date'] = year_month
    all_dates_map = all_dates_map.append(all_types, ignore_index=True)

In [ ]:
output_df = pd.merge(all_dates_map, channel_df, on=['Date', 'Brand', 'Province', 'Cost_Type_2'], how='left')
output_df = output_df.fillna(0)

## Output result

In [ ]:
def get_level_5(cost_type):
    if cost_type == 'Sales KA + KMBS':
        return 'KA'
    elif cost_type == 'Sales Online':
        return 'EC'
    elif cost_type == 'Sales GT':
        return 'GT'
    elif cost_type == 'Sales RKMBS':
        return 'RKMBS'
    return cost_type


def format_yearmonth(year_month):
    return time.strftime('%Y/%m/%d', time.strptime(year_month,'%Y%m'))

output_df['Date'] = output_df['Date'].apply(lambda x : format_yearmonth(x))

output_df['Level 1'] = 'Total A&P'
output_df['Level 2'] = 'Total Channel'
output_df['Level 3'] = 'Total P Cost'
output_df['Level 4'] = 'Total Channel'
output_df['Level 5'] = output_df['Cost_Type_2'].apply(lambda x : get_level_5(x))

output_df = output_df[['Date', 'Brand', 'Province', 'Level 1', 'Level 2', 'Level 3', 'Level 4', 'Level 5', 'Value']]

In [ ]:
output_file_path = config['project_path'] + '/' \
                        + config['temp_folder_path'] + '/' \
                        + config['channel_output'] 

output_df.to_excel(output_file_path, index=False)